In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from millify import millify
from pathlib import Path

In [2]:

p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: {stringpath}")
ngsimfile=stringpath + '/' + 'Cleaned_NGSIM_Data_Part_1.csv'

ngsim = pd.read_csv(ngsimfile,low_memory=True)


original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


In [3]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim['calculated_length'] = np.NaN


filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) ]
#These two vehicles have a bad trajectory and keep on following each other in a single lane in a matter of 
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') & ( ((ngsim['Vehicle_ID'] != 1748 )  & (ngsim['Preceding'] != 1734 )) &(  (ngsim['Vehicle_ID'] != 1734 )  & (ngsim['Preceding'] != 1748 )))]
#filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80')]

In [4]:
#filtered_ngsim_U['preceding_v_length'] = filtered_ngsim_U.apply(lambda x: x['v_length']['Preceding'])
#filtered_ngsim_U['space_headway_rear_to_front1'] = filtered_ngsim_U.apply(lambda x: x['v_length']['Preceding'].mean())

In [5]:
filtered_ngsim_U.head()

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,part1_for_loop_output_mean_preceding_vehicle_length,v_Class_Name,Vehicle_combination,calculated_length
0,515,2330,1118848075000,30.034,188.062,6451203.729,1873252.549,3.9624,2,7.104888,...,5.11,us-101,1117685055901,Car,30.916880,4.351494,5.3848,Car,Car-Car,NaN
1,2127,6459,1118847624800,19.632,1775.614,6452425.122,1872172.475,4.1148,2,11.436096,...,1.30,us-101,1117684605701,Car,10.872216,0.950693,4.0386,Car,Car-Car,NaN
2,1033,4827,1118848324700,6.202,1701.144,6452347.673,1872258.452,4.1148,2,12.798552,...,0.92,us-101,1117685305601,Car,7.257288,0.567040,4.5720,Car,Car-Car,NaN
3,1890,9157,1118849672700,53.514,817.521,6451655.238,1872800.663,7.3152,3,13.752576,...,2.27,us-101,1117686653601,Car,27.325320,1.986924,3.9624,Heavy Vehicle,Car-Heavy Vehicle,NaN
4,744,3392,1118848181200,28.878,490.086,6451422.353,1873041.018,4.5720,2,7.479792,...,1.54,us-101,1117685162101,Car,6.746240,0.901929,4.7752,Car,Car-Car,NaN


In [6]:
#filtered_ngsim_U['trial_mean_length_Current_vehicle']=filtered_ngsim_U.groupby(['Vehicle_ID'])['v_length'].mean()
filtered_ngsim_U['trial_mean_length_Current_vehicle']=filtered_ngsim_U.groupby(['Vehicle_ID'])['v_length'].agg(lambda x: x.unique().mean())


In [7]:
filtered_ngsim_U=filtered_ngsim_U.assign(trial_previous_Vehicle_length=filtered_ngsim_U['trial_mean_length_Current_vehicle'].loc[filtered_ngsim_U['Preceding']].values)
filtered_ngsim_U['trial_verify_space_headway']=filtered_ngsim_U['Space_Headway'] - filtered_ngsim_U['trial_previous_Vehicle_length']
filtered_ngsim_U

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,part1_for_loop_output_mean_preceding_vehicle_length,v_Class_Name,Vehicle_combination,calculated_length,trial_mean_length_Current_vehicle,trial_previous_Vehicle_length,trial_verify_space_headway
0,515,2330,1118848075000,30.034,188.062,6451203.729,1873252.549,3.9624,2,7.104888,...,Car,30.916880,4.351494,5.3848,Car,Car-Car,NaN,NaN,5.3848,30.916880
1,2127,6459,1118847624800,19.632,1775.614,6452425.122,1872172.475,4.1148,2,11.436096,...,Car,10.872216,0.950693,4.0386,Car,Car-Car,NaN,9.4488,4.0386,10.872216
2,1033,4827,1118848324700,6.202,1701.144,6452347.673,1872258.452,4.1148,2,12.798552,...,Car,7.257288,0.567040,4.5720,Car,Car-Car,NaN,5.2324,4.5720,7.257288
3,1890,9157,1118849672700,53.514,817.521,6451655.238,1872800.663,7.3152,3,13.752576,...,Car,27.325320,1.986924,3.9624,Heavy Vehicle,Car-Heavy Vehicle,NaN,4.8768,3.9624,27.325320
4,744,3392,1118848181200,28.878,490.086,6451422.353,1873041.018,4.5720,2,7.479792,...,Car,6.746240,0.901929,4.7752,Car,Car-Car,NaN,4.7244,4.7752,6.746240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4098928,1741,7118,1118848553800,18.614,1439.138,6452142.330,1872421.454,3.9624,2,6.096000,...,Car,12.594336,2.066000,3.8100,Car,Car-Car,NaN,NaN,3.8100,12.594336
4098929,601,1871,1118847166000,54.735,1988.482,6452564.992,1872006.298,5.4864,2,17.614392,...,Car,30.342840,1.722616,5.1816,Car,Car-Car,NaN,NaN,5.1816,30.342840
4098930,1752,6989,1118848540900,20.360,1271.352,6452014.737,1872530.517,5.0292,2,3.645408,...,Car,7.015480,1.924470,4.2164,Car,Car-Car,NaN,NaN,4.2164,7.015480
4098931,2634,8057,1118847784600,40.871,1617.040,6452292.021,1872261.001,4.4196,2,7.620000,...,Car,10.863072,1.425600,4.2672,Car,Car-Car,NaN,NaN,4.2672,10.863072


In [8]:
filtered_ngsim_U.to_csv('verify.csv', index = False)

In [9]:
#filtered_ngsim_U.apply(space_headway_rear_to_front1= lambda x: filtered_ngsim_U['v_length'][filtered_ngsim_U['Vehicle_ID']== filtered_ngsim_U['Preceding'][x]].mean())

In [10]:
#filtered_ngsim_U['mean_length']
#filtered_ngsim_U[filtered_ngsim_U['Vehicle_ID'] ==515].to_csv('515file.csv', index = False)
#filtered_ngsim_I['mean_vehicle_length']=filtered_ngsim_I.groupby(['Vehicle_ID'])['v_length'].agg(lambda x: x.unique().mean(skipna=True))

In [ ]:
#filtered_ngsim_U[filtered_ngsim_U['Vehicle_ID'] ==515]
#filtered_ngsim_U
#x[x['Vehicle_ID']==515]
#filtered_ngsim_U['mean_vehicle_length']=filtered_ngsim_U.groupby(['Vehicle_ID'])['v_length'].agg(lambda x: x.unique().mean())
#dict_x.keys()
#dict_x[500]
#dict_x[515]
#vehicle_lengths_U[vehicle_lengths_U['Vehicle_ID']==500]
#dict_x[500]

In [ ]:
#filtered_ngsim_U=filtered_ngsim_U.assign(previous_Vehicle_length=filtered_ngsim_U['mean_vehicle_length'].loc[filtered_ngsim_U['Preceding']].values)
#filtered_ngsim_I=filtered_ngsim_I.assign(previous_Vehicle_length=filtered_ngsim_I['mean_vehicle_length'].loc[filtered_ngsim_I['Preceding']].values)